In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from html import unescape
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import sent_tokenize, word_tokenize
import seaborn as sns
import datetime as datetime
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
talks = pd.read_pickle('../datasets/videosData.pkl')
talks.head(2)

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,videoDescription,currentDate,publishDate,text,nouns
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,7 months ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,Nikit talks about wildlife and his work within...,1.57697e+09,2019-05-17 00:00:00,I don't have never was in Mumbai so muchall of...,mumbai muchall cause three-second ofthe pinto ...
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,7 months ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,2019-05-17 00:00:00,I always loved the Mediterranean maybe because...,mediterranean north phrase feride scenography ...


In [10]:
len(talks)

78391

In [3]:
import pickle
with open('../models/vectorsTalks.pkl', 'rb') as pkl_vec:
    vectorizer = pickle.load(pkl_vec)
X = vectorizer.transform(talks.nouns)

In [4]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(X[0], X).flatten()
cosine_similarities.argsort()[:-5:-1]

array([    0, 46309,  8422, 26672], dtype=int64)

In [5]:
talks.iloc[cosine_similarities.argsort()[:-5:-1]]

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,videoDescription,currentDate,publishDate,text,nouns
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,7 months ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,Nikit talks about wildlife and his work within...,1.57697e+09,2019-05-17 00:00:00,I don't have never was in Mumbai so muchall of...,mumbai muchall cause three-second ofthe pinto ...
STvYyCjLvfU,How to travel ethically?,Innocent Mutanga,TEDxCUHK,STvYyCjLvfU,How to travel ethically? | Innocent Mutanga | ...,3 months ago,21,2,1344,"[TEDxTalks, English, Global Issues, Globalizat...",680,Growing up in Zimbabwe at the tourist town of ...,1.57792e+09,2019-09-17 00:00:00,[Applause]okay and you guys see me alright ah ...,[ applause ] okay thing littletoo dark lights ...
IQfkFu5gPC0,How to be a Catalyst of Change for your society,Anirudh S Dutt,TEDxGlobalAcademy,IQfkFu5gPC0,How to be a Catalyst of Change for your societ...,1 year ago,153,2,1024,"[TEDxTalks, English, Global Issues, Environmen...",2268,"Anirudh Dutt, Founder and President of Let’s B...",1.57699e+09,2018-05-23 00:00:00,Eldar bounnam's Cara a very good eveningto one...,eldar bounnam cara eveningto heatley whatis sa...
ru8EMz4arWM,Yuva Ignited Minds,Mayank Solanki,TEDxSonaCollege,ru8EMz4arWM,Yuva Ignited Minds: Mayank Solanki at TEDxSona...,6 years ago,11,0,1097,"[TEDxSonaCollege, NGO, ted talks, tedx, Social...",826,Mayank Solanki is the co-founder of the NGO Yu...,1.57699e+09,2012-06-21 00:00:00,I have to first thank God because andalso ment...,god andalso name couple magician mics share sa...


In [6]:
talks.columns

Index(['title', 'speaker', 'event', 'videoId', 'titleRaw', 'publishedTimeRaw',
       'likes', 'dislikes', 'lengthSeconds', 'keywords', 'views',
       'videoDescription', 'currentDate', 'publishDate', 'text', 'nouns'],
      dtype='object')

In [7]:
cosine_similarities.argsort()[:20:1]

array([72689, 77510,  2765, 56510, 66417, 33150, 63903, 47304, 47305,
       68012, 47308, 78333, 33922, 11968,  4259,  6733, 34246, 42422,
        5953, 66050], dtype=int64)

In [8]:
toFind = vectorizer.transform([talks.loc['L5sBdR4-GGM'].nouns])[0]

cosine_similarities = linear_kernel(toFind, X).flatten()

for id in talks.iloc[cosine_similarities.argsort()[:-20:-1]].index:
    print(talks.loc[id].titleRaw)
    print('https://www.youtube.com/watch?v='+id, '\n')

Are Games Art? | Brenda Romero | TEDxGalway
https://www.youtube.com/watch?v=L5sBdR4-GGM 

Games with Purpose | Phil Stuart | TEDxKingAlfredSchool
https://www.youtube.com/watch?v=-9dSrH6tu4s 

The power of play: Bas van Haren at TEDxHanzeUniversity
https://www.youtube.com/watch?v=NyfIFMGj2yY 

What if...games could do good?: Stephen Knightly at TEDxAuckland
https://www.youtube.com/watch?v=QhY7gAXpVD8 

How games are changing the way we age | Bob De Schutter | TEDxMiamiUniversity
https://www.youtube.com/watch?v=ZfU6DGfhWt8 

Computer Games for a Better Life | Rainer Malaka | TEDxOsnabrückUniversity
https://www.youtube.com/watch?v=AEhz0MUfd7Q 

The Good of The Video Gaming Life | Ahmed ElKarim Ahmed | TEDxYouth@GAA
https://www.youtube.com/watch?v=DjHElcpoZe4 

Video-games in art and education | Victor Bethonico Foresti de Oliveira Castro | TEDxYouth@EAB
https://www.youtube.com/watch?v=g8v-olDxs4c 

Games Are Serious Fun: Mark Rein-Hagen at TEDxTbilisi
https://www.youtube.com/watch?v=KSo0O

In [9]:
talks.loc['nWecIwtN2ho']

title                                                         Actitud
speaker                                                Victor Küppers
event                                              TEDxAndorralaVella
videoId                                                   nWecIwtN2ho
titleRaw                Actitud | Victor Küppers | TEDxAndorralaVella
publishedTimeRaw                                          6 years ago
likes                                                          101249
dislikes                                                         4204
lengthSeconds                                                    1204
keywords            [Andorra, Spanish, TEDx, TEDxAndorralaVella, t...
views                                                         7768520
videoDescription    Never miss a talk! SUBSCRIBE to the TEDx chann...
currentDate                                               1.57697e+09
publishDate                                       2013-10-12 00:00:00
text                